<a href="https://colab.research.google.com/github/rahuljungbahadur/hands_on_ml_book/blob/main/chp13_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
os.listdir()

['.config', 'sample_data']

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving aclImdb_v1.tar.gz to aclImdb_v1.tar.gz


NameError: ignored

In [5]:
os.listdir()

['.config', 'aclImdb_v1.tar.gz', 'sample_data']

In [6]:
import tarfile

# open file
file = tarfile.open('aclImdb_v1.tar.gz')
  
# print file names
print(file.getnames())
  
# extract files
file.extractall('./imdb_dataset')
  
# close file
file.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
os.listdir('imdb_dataset/aclImdb/train/')

['neg',
 'urls_pos.txt',
 'urls_neg.txt',
 'labeledBow.feat',
 'unsupBow.feat',
 'urls_unsup.txt',
 'unsup',
 'pos']

In [67]:
trainSet_posFp = 'imdb_dataset/aclImdb/train/pos'
trainSet_negFp = 'imdb_dataset/aclImdb/train/neg'

In [17]:
import tensorflow as tf

In [31]:
help(tf.data.Dataset.list_files)

Help on function list_files in module tensorflow.python.data.ops.dataset_ops:

list_files(file_pattern, shuffle=None, seed=None, name=None)
    A dataset of all files matching one or more glob patterns.
    
    The `file_pattern` argument should be a small number of glob patterns.
    If your filenames have already been globbed, use
    `Dataset.from_tensor_slices(filenames)` instead, as re-globbing every
    filename with `list_files` may result in poor performance with remote
    storage systems.
    
    Note: The default behavior of this method is to return filenames in
    a non-deterministic random shuffled order. Pass a `seed` or `shuffle=False`
    to get results in a deterministic order.
    
    Example:
      If we had the following files on our filesystem:
    
        - /path/to/dir/a.txt
        - /path/to/dir/b.py
        - /path/to/dir/c.py
    
      If we pass "/path/to/dir/*.py" as the directory, the dataset
      would produce:
    
        - /path/to/dir/b.py
    

In [111]:
trainSet_posFp_random = tf.data.Dataset.list_files(file_pattern='imdb_dataset/aclImdb/train/pos/*.txt')
trainSet_negFp_random = tf.data.Dataset.list_files(file_pattern='imdb_dataset/aclImdb/train/neg/*.txt')

In [79]:
os.listdir('imdb_dataset/aclImdb')

['README', 'imdbEr.txt', 'valSet', 'testSet', 'test', 'imdb.vocab', 'train']

In [76]:
# os.mkdir('./imdb_dataset/aclImdb/testSet/neg')
os.mkdir('./imdb_dataset/aclImdb/testSet/pos')
os.mkdir('./imdb_dataset/aclImdb/valSet/neg')
os.mkdir('./imdb_dataset/aclImdb/valSet/pos')

### Split the test set into a validation set (15,000) and a test set (10,000).

In [ ]:
import shutil

## test set
[shutil.copy2(f'imdb_dataset/aclImdb/test/neg/{fp}', 'imdb_dataset/aclImdb/testSet/neg') for fp in os.listdir('imdb_dataset/aclImdb/test/neg')[:5000]]
[shutil.copy2(f'imdb_dataset/aclImdb/test/pos/{fp}', 'imdb_dataset/aclImdb/testSet/pos') for fp in os.listdir('imdb_dataset/aclImdb/test/pos')[:5000]]

## Valid Set
[shutil.copy2(f'imdb_dataset/aclImdb/test/neg/{fp}', 'imdb_dataset/aclImdb/valSet/neg') for fp in os.listdir('imdb_dataset/aclImdb/test/neg')[5000:]]
[shutil.copy2(f'imdb_dataset/aclImdb/test/pos/{fp}', 'imdb_dataset/aclImdb/valSet/pos') for fp in os.listdir('imdb_dataset/aclImdb/test/pos')[5000:]]



In [97]:
(len(os.listdir('imdb_dataset/aclImdb/valSet/pos/')), len(os.listdir('imdb_dataset/aclImdb/valSet/neg/')))

(7500, 7500)

In [98]:
(len(os.listdir('imdb_dataset/aclImdb/testSet/pos/')), len(os.listdir('imdb_dataset/aclImdb/testSet/neg/')))

(5000, 5000)

#### Qns 2 Done

## Use tf.data to create an efficient dataset for each set.

In [ ]:
help(tf.data.TextLineDataset)

In [138]:
trainSet_pos = tf.data.TextLineDataset(trainSet_posFp_random, num_parallel_reads=5).batch(1)
trainSet_neg = tf.data.TextLineDataset(trainSet_negFp_random, num_parallel_reads=5).batch(1)

In [139]:
testSet_pos = tf.data.TextLineDataset(tf.data.Dataset.list_files('imdb_dataset/aclImdb/testSet/pos/'), num_parallel_reads=5).batch(1)
testSet_neg = tf.data.TextLineDataset(tf.data.Dataset.list_files('imdb_dataset/aclImdb/testSet/neg/'), num_parallel_reads=5).batch(1)

In [140]:
valSet_pos = tf.data.TextLineDataset(tf.data.Dataset.list_files('imdb_dataset/aclImdb/valSet/pos/'), num_parallel_reads=5).batch(1)
valSet_neg = tf.data.TextLineDataset(tf.data.Dataset.list_files('imdb_dataset/aclImdb/valSet/neg/'), num_parallel_reads=5).batch(1)

## Qns 4 Create a binary classification model, using a TextVectorization layer

In [131]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization, Dense, Input

In [ ]:
help(TextVectorization)

In [175]:
model1 = Sequential()

model1.add(Input(shape=(1,), dtype=tf.string))




In [155]:
tf_layer = TextVectorization(output_mode='int')

In [156]:
tf_layer.adapt(trainSet_pos)

In [176]:
model1.add(tf_layer)

In [163]:
sample_file = []
with open(f'{trainSet_posFp}/7493_7.txt') as file:
  sample_file=file.read()

In [177]:
sample_file.count('')

2831

In [178]:
model1.predict([sample_file]).size

515